## 모델 분석해보기

``` python
이제 직접 모델을 만들어낼 줄 알게 되었습니다!

모델을 만들고 난 후 어떤 module과 parameter를 썼는지
어떻게 알 수 있는 거죠?

느낌상 "nn.Module" Documentation에 그 방법이 나와있을 것 같아요!

제가 전에 작성해놓았던 코드를 가져왔어요! 그때 보셨을 때와 조금 다르죠?
배운 것을 적용해보느라고 조금씩 수정을 해보았어요!
제가 만든 이 모델을 함께 분석해봐요!
```

- [Documentation main - PyTorch 공식 문서](https://pytorch.org/docs/stable/index.html)
- [torch.nn.Module - PyTorch 공식 문서](https://pytorch.org/docs/stable/generated/torch.nn.Module.html)

In [5]:
import torch
from torch import nn
from torch.nn.parameter import Parameter


# 아래 코드는 수정하실 필요가 없습니다!
# 하지만 아래 과제를 진행하기 전에 아래 코드를 보면서 최대한 이해해보세요!

# Function
class Function_A(nn.Module):
    def __init__(self, name):
        super().__init__()  # 부모클래스 상속
        self.name = name # name을 받고는 있지만 별 의미는 없다.

    def forward(self, x):
        x = x * 2 # forward로는 x만 사용한다.
        return x

class Function_B(nn.Module):
    def __init__(self):
        super().__init__()
        self.W1 = Parameter(torch.Tensor([10])) # 파라미터 클래스 객체에 저장.
        self.W2 = Parameter(torch.Tensor([2])) # 클래스 = 대문자로 시작, 메서드는 소문자로 시작.

    def forward(self, x):
        x = x / self.W1
        x = x / self.W2

        return x

class Function_C(nn.Module):
    def __init__(self):
        super().__init__()
        self.register_buffer('duck', torch.Tensor([7]), persistent=True) # duck 이라는 이름으로 tensor 7을 기억할 거야.

    def forward(self, x):
        x = x * self.duck 
        
        return x

class Function_D(nn.Module):
    def __init__(self):
        super().__init__()
        self.W1 = Parameter(torch.Tensor([3]))
        self.W2 = Parameter(torch.Tensor([5]))
        self.c = Function_C()

    def forward(self, x):
        x = x + self.W1
        x = self.c(x)
        x = x / self.W2

        return x


# Layer
class Layer_AB(nn.Module):
    def __init__(self):
        super().__init__()

        self.a = Function_A('duck')
        self.b = Function_B()

    def forward(self, x):
        x = self.a(x) / 5
        x = self.b(x)

        return x

class Layer_CD(nn.Module):
    def __init__(self):
        super().__init__()

        self.c = Function_C()
        self.d = Function_D()

    def forward(self, x):
        x = self.c(x)
        x = self.d(x) + 1

        return x


# Model
class Model(nn.Module):
    def __init__(self):
        super().__init__()

        self.ab = Layer_AB()
        self.cd = Layer_CD()

    def forward(self, x):
        x = self.ab(x)
        x = self.cd(x)

        return x

x = torch.tensor([7])

model = Model()
model(x)

tensor([6.5720], grad_fn=<AddBackward0>)

### named_children vs named_modules

``` python
제가 만든 모델에서 어떤 module들이 있었는지 기억이 나질 않아요!
그래서 모델 내부의 module들 목록을 보고싶어요!

Documentation을 찾아보니까 children이나 module이라는 이름을 가진
함수가 바로 제가 원하는 기능을 가진 것 같아요!

하지만 이 둘은 무슨 차이일까요?
역시 직접 코드를 통해서 확인해봐야겠어요!
```

- [named_children - PyTorch 공식 문서](https://pytorch.org/docs/stable/generated/torch.nn.Module.html?highlight=child#torch.nn.Module.named_children)
- [named_modules - PyTorch 공식 문서](https://pytorch.org/docs/stable/generated/torch.nn.Module.html?highlight=named#torch.nn.Module.named_modules)


In [2]:
for name, module in model.named_modules():
    print(f"[ Name ] : {name}\n[ Module ]\n{module}")
    print("-" * 30)

[ Name ] : 
[ Module ]
Model(
  (ab): Layer_AB(
    (a): Function_A()
    (b): Function_B()
  )
  (cd): Layer_CD(
    (c): Function_C()
    (d): Function_D(
      (c): Function_C()
    )
  )
)
------------------------------
[ Name ] : ab
[ Module ]
Layer_AB(
  (a): Function_A()
  (b): Function_B()
)
------------------------------
[ Name ] : ab.a
[ Module ]
Function_A()
------------------------------
[ Name ] : ab.b
[ Module ]
Function_B()
------------------------------
[ Name ] : cd
[ Module ]
Layer_CD(
  (c): Function_C()
  (d): Function_D(
    (c): Function_C()
  )
)
------------------------------
[ Name ] : cd.c
[ Module ]
Function_C()
------------------------------
[ Name ] : cd.d
[ Module ]
Function_D(
  (c): Function_C()
)
------------------------------
[ Name ] : cd.d.c
[ Module ]
Function_C()
------------------------------


In [3]:
for name, child in model.named_children():
    print(f"[ Name ] : {name}\n[ Children ]\n{child}")
    print("-" * 30)

[ Name ] : ab
[ Children ]
Layer_AB(
  (a): Function_A()
  (b): Function_B()
)
------------------------------
[ Name ] : cd
[ Children ]
Layer_CD(
  (c): Function_C()
  (d): Function_D(
    (c): Function_C()
  )
)
------------------------------


``` python
아하! 이제 알겠어요!

"children"은 한 단계 아래의 submodule까지만 표시하는 것이고
"modules"는 자신에게 속하는 모든 submodule들을 표시해주는 것이군요!

"named_modules", "named_children"은 module의 이름도 돌려주는데
그냥 module만 필요한 경우는 "modules", "children"를 사용하면 되겠네요!

아니 그런데 왜 "Function_D"가 "Function_C"를 참조하는 중이죠?
같은 기본 단위는 서로 독립적이어야 하는데 이러면 문제가 있을 것 같아요!
분석이 끝나면 수정을 해야겠어요!
```

### get_submodule

``` python
제가 만든 모델 내부에 어떤 module들이 있는지 잘 알겠어요!
이제 제가 원하는 특정 module만을 가져오고 싶어요!

Function_A를 가져다 줄 수 있을까요?
```

- [get_submodule - PyTorch 공식 문서](https://pytorch.org/docs/stable/generated/torch.nn.Module.html?highlight=get_submodule#torch.nn.Module.get_submodule)


In [8]:
# TODO : Function_A 를 가져오세요!
submodule = model.get_submodule('ab.a')  # Function_A 의 이름을 str 타입으로 호출


# 아래 코드는 수정하실 필요가 없습니다!
if submodule.__class__.__name__  == 'Function_A':
    print("🎉🎉🎉 성공!!! 🎉🎉🎉")
else:
    print("다시 도전해봐요!")

🎉🎉🎉 성공!!! 🎉🎉🎉


### Parameter

``` python
이제 module에 대한 자신감이 생겼어요!
그런데 문득 생각해보니까 Parameter에 대해서 공부하면서
제가 어떤 모듈에 Parameter를 생성해놨던 것 같아요!

저희가 module에서 module 목록을 보고 특정 module을
찾기도 했던 것처럼 Parameter에서도 똑같이 해봐요!
```

- [parameters - PyTorch 공식 문서](https://pytorch.org/docs/stable/generated/torch.nn.Module.html?highlight=parameters#torch.nn.Module.parameters)
- [named_parameters - PyTorch 공식 문서](https://pytorch.org/docs/stable/generated/torch.nn.Module.html?highlight=named#torch.nn.Module.named_parameters)
- [get_parameter - PyTorch 공식 문서](https://pytorch.org/docs/stable/generated/torch.nn.Module.html?highlight=get#torch.nn.Module.get_parameter)


In [ ]:
# 4개의 Parameter를 만들었었군요!
for name, parameter in model.named_parameters():
    print(f"[ Name ] : {name}\n[ Parameter ]\n{parameter}")
    print("-" * 30)

``` python
아하! 제가 4개의 Parameter를 만들었었네요!
이름을 표시하니까 Parameter가 어디에 속하는지 알 수 있어 너무 좋아요!

- "Function_B"에 W1, W2 Parameter 2개
- "Function_D"에 W1, W2 Parameter 2개

"parameters"를 사용해서 목록을 확인해도 되지만
이름이 표시 안되니까 어떤 module에 속한
Parameter인지 알기가 너무 힘들 것 같아요!

Function_B에 속해있는 Parameter W1을 사용하고 싶은데
이를 가져다 주실수 있나요?
```

In [9]:
# TODO : Function_B에 속하는 Parameter W1을 가져오세요!
parameter = model.get_parameter('ab.b.W1') # 모든 파라미터가 미분이 가능해야 딥러닝이 가능하다는 사실, 알고 계셨나요?


parameter # 10

Parameter containing:
tensor([10.], requires_grad=True)

### Buffer

``` python
생각을 해보니까 buffer도 하나 추가해뒀던 것 같아요!
buffer도 같이 분석해봐요!
```

- [buffers - PyTorch 공식 문서](https://pytorch.org/docs/stable/generated/torch.nn.Module.html?highlight=buffers#torch.nn.Module.buffers)
- [named_buffers - PyTorch 공식 문서](https://pytorch.org/docs/stable/generated/torch.nn.Module.html?highlight=named_buffers#torch.nn.Module.named_buffers)
- [get_buffer - PyTorch 공식 문서](https://pytorch.org/docs/stable/generated/torch.nn.Module.html?highlight=get_buffer#torch.nn.Module.get_buffer)


In [12]:
# TODO : named_buffers를 사용해서 model에 속하는 buffer 전체 목록을 가져오세요!
# 이름 ㅇ 
for name, buffer in model.named_buffers():
    print(f"[ Name ] : {name}\n[ Buffer ] : {buffer}")
    print("-" * 30)

[ Name ] : cd.c.duck
[ Buffer ] : tensor([7.])
------------------------------
[ Name ] : cd.d.c.duck
[ Buffer ] : tensor([7.])
------------------------------


In [13]:
# TODO : buffers를 사용해서 model에 속하는 buffer 전체 목록을 가져오세요!
# 이름 x
for buffer in model.buffers():
    print(f"[ Buffer ] : {buffer}")
    print("-" * 30)

[ Buffer ] : tensor([7.])
------------------------------
[ Buffer ] : tensor([7.])
------------------------------


In [14]:
# TODO : Function_C에 속하는 Buffer를 가져오세요!
buffer = model.get_buffer('cd.c.duck')


buffer # 7

tensor([7.])

### 모델 수정하기 - Docstring 작성

``` python
가볍게 Docstring을 작성해봐요!
Numpy, Pydoc, Google 등 다양한 스타일이 있으니까 궁금하면
아래 링크에서 읽어보면서 공부해보도록 해요!

지금은 스타일 상관없이 Docstring이라는 것을 추가하기만 해보죠!
```

- [Documentation main - PyTorch 공식 문서](https://pytorch.org/docs/stable/index.html)

**✨ 유용한 자료 ✨**
- [Docstrings in Python - Data Camp](https://www.datacamp.com/community/tutorials/docstrings-python)

In [15]:
import torch
from torch import nn
from torch.nn.parameter import Parameter

# TODO : Docstring을 추가해보세요!
class Model(nn.Module):
    """
    class 의 문서화 내용을 입력합니다.
    docstring은 모듈, 함수, 클래스 또는 메소드 정의의 첫 번째 명령문으로 발생하는 문자열 리터럴

    """
    def __init__(self):
        super().__init__()



# 아래 코드는 수정하실 필요가 없습니다!
model = Model()

if model.__doc__: # 이렇게 __doc__ 을 통해 속성으로서 접근할 수 있음.
    print("🎉🎉🎉 성공!!! 🎉🎉🎉")
else:
    print("다시 도전해봐요!")

🎉🎉🎉 성공!!! 🎉🎉🎉


### BatchNorm1d 분석해보기

``` python
PyTorch가 미리 만들어둔 module중 하나인 BatchNorm1d로 분석해봅시다!
```

- [Documentation main - PyTorch 공식 문서](https://pytorch.org/docs/stable/index.html)
- [torch.nn.BatchNorm1d - PyTorch 공식 문서](https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm1d.html#torch.nn.BatchNorm1d)

In [17]:
import torch
import torch.nn

module = nn.BatchNorm1d(10)

# TODO : nn.BatchNorm1d의 parameter와 buffer 갯수를 알아내세요!
parameter_n = len(list(module.parameters()))
buffer_n = len(list(module.buffers()))

# TODO : nn.BatchNorm1d의 buffer 이름을 알아내세요!
#        [이름, 이름, 이름] 형태로 저장해주세요!
buffer_names = [name for name, _ in module.named_buffers()]



# 아래 코드는 수정하실 필요가 없습니다!
answer = set(['running_mean', 'running_var', 'num_batches_tracked'])

if parameter_n == 2 and buffer_n == 3 and answer == set(buffer_names):
    print("🎉🎉🎉 성공!!! 🎉🎉🎉")
else:
    print("다시 도전해봐요!")

🎉🎉🎉 성공!!! 🎉🎉🎉
